In [1]:
import numpy as np
import numba as nb
import numpy.random as rng

In [2]:
mus = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])

In [3]:
@nb.njit
def exponential(scales):
    
    length = len(scales)
    t_array = np.zeros((length))
    
    for i in range(length):
        t_array[i] = rng.exponential(scales[i])
        
    return t_array
# this can be written as 
# @nb.vectorize
# def exponential(scale):
#     return rng.exponential(scale)


# Using external vectorized np.random.exponential

In [4]:
@nb.njit
def rand(mus, step_max, seed):
    
    # tuple = lambda x: to_fixed_tuple(x, max_size)
    # exponential = lambda scales: np.array([rng.exponential(x) for x in scales] ) 
    
    step = 0
    tau = 0.
    
    rng.seed(seed)
    dtaus = np.zeros(len(mus))
    
    while (step < step_max):
        step = step + 1
        
        dtaus = exponential(1./mus)
        
        tau += np.min(dtaus)
        
    return tau
    

In [5]:
@nb.njit
def nopar_simulate_multi(mus, step_max, num):
    
    seeds = np.arange(num)
    taus = np.zeros(num)
    
    for i in nb.prange(num):
        tau = rand(mus, step_max, seeds[i])
        
        taus[i] = tau
        
        
    return taus
        

In [6]:
@nb.njit(parallel = True)
def par_simulate_multi(mus, step_max, num):
    
    seeds = np.arange(num)
    taus = np.zeros(num)
    
    for i in nb.prange(num):
        tau = rand(mus, step_max, seeds[i])
        
        taus[i] = tau
        
        
    return taus

In [7]:
# compile
tmp = nopar_simulate_multi(mus, 2, 2)
tmp = par_simulate_multi(mus, 2, 2)


In [8]:
%%time
tmp = nopar_simulate_multi(mus, 50000, 320)

CPU times: user 10.2 s, sys: 9.47 ms, total: 10.2 s
Wall time: 10.1 s


In [9]:
%%time
tmp = par_simulate_multi(mus, 50000, 320)

CPU times: user 6min 39s, sys: 306 ms, total: 6min 39s
Wall time: 7.16 s


# Without calling external funciton

In [10]:
@nb.njit
def rand_wo(mus, step_max, seed):
    
    # tuple = lambda x: to_fixed_tuple(x, max_size)
    # exponential = lambda scales: np.array([rng.exponential(x) for x in scales] ) 
    
    step = 0
    tau = 0.
    
    rng.seed(seed)
    dtaus = np.zeros(len(mus))
    
    while (step < step_max):
        step = step + 1
        
        for i in range(len(mus)):
            dtaus[i] = rng.exponential(1./mus[i])
        
        
        tau += np.min(dtaus)
        
    return tau
    

In [11]:
@nb.njit
def nopar_simulate_multi_wo(mus, step_max, num):
    
    seeds = np.arange(num)
    taus = np.zeros(num)
    
    for i in nb.prange(num):
        tau = rand_wo(mus, step_max, seeds[i])
        
        taus[i] = tau
        
        
    return taus
        

In [12]:
@nb.njit(parallel = True)
def par_simulate_multi_wo(mus, step_max, num):
    
    seeds = np.arange(num)
    taus = np.zeros(num)
    
    for i in nb.prange(num):
        tau = rand_wo(mus, step_max, seeds[i])
        
        taus[i] = tau
        
        
    return taus

In [13]:
# compile
tmp = nopar_simulate_multi_wo(mus, 2, 2)
tmp = par_simulate_multi_wo(mus, 2, 2)


In [14]:
%%time
tmp = nopar_simulate_multi_wo(mus, 50000, 320)

CPU times: user 7.66 s, sys: 3.05 ms, total: 7.66 s
Wall time: 7.6 s


In [15]:
%%time
tmp = par_simulate_multi_wo(mus, 50000, 320)

CPU times: user 9.45 s, sys: 25 ms, total: 9.48 s
Wall time: 220 ms
